In [18]:
import scrapy
import requests
from scrapy.http import TextResponse
from fake_useragent import UserAgent

# Crawling Poject : NAVER MOVIE Data

- 네이버의 영화 장르별 데이터 크롤링
    - 네이버 영화디렉토리 URL 
        - 네이버영화디렉토리 페이지 URL : https://movie.naver.com/movie/sdb/browsing/bmovie.nhn?genre=19
        
- 수집 카테고리 : 애니메이션, 액션, 판타지, 드라마, 다큐멘터리, 공포, 에로, 코미디
    - 총 115150편
    - 영화제목, 주요정보, 별점, 댓글:최근 1~5페이지

In [19]:
!rm -rf navermovie

In [20]:
!scrapy startproject navermovie

New Scrapy project 'navermovie', using template directory '/Users/jhkim/opt/miniconda3/lib/python3.7/site-packages/scrapy/templates/project', created in:
    /Users/jhkim/Documents/dev/TIL_/navermovie

You can start your first spider with:
    cd navermovie
    scrapy genspider example example.com


In [33]:
%%writefile navermovie/navermovie/items.py
import scrapy


class NavermovieItem(scrapy.Item):
    title = scrapy.Field()
    story = scrapy.Field()
    ntz_star = scrapy.Field()
    review = scrapy.Field()

Overwriting navermovie/navermovie/items.py


In [22]:
url = "https://movie.naver.com/movie/sdb/browsing/bmovie.nhn?genre=19&page=1"

req = requests.get(url)
response = TextResponse(req.url, body=req.text, encoding="utf-8")

# url = "https://movie.naver.com/movie/sdb/browsing/bmovie.nhn?genre=19&page=1"
# headers = {"User-Agent": UserAgent().chrome,
#            "cookie": "NRTK=ag#all_gr#1_ma#-2_si#0_en#0_sp#0; NNB=YEPGKPQZBHZV4; nx_ssl=2; MM_NEW=1; NFS=2; MM_NOW_COACH=1; NDARK=Y; _ga=GA1.1.233025110.1593324245; _ga_7VKFYR6RV1=GS1.1.1594610477.8.1.1594610507.30; BMR=s=1594882753112&r=https%3A%2F%2Fm.blog.naver.com%2FPostView.nhn%3FblogId%3Dssdyka%26logNo%3D221494190411%26proxyReferer%3Dhttps%3A%252F%252Fwww.google.com%252F&r2=https%3A%2F%2Fwww.google.com%2F; NM_THUMB_PROMOTION_BLOCK=Y; page_uid=UxnXadprvxssshvf9Vhssssst8V-486356; csrf_token=4a4558ed-1707-4101-a136-e1a770218613; JSESSIONID=6E2DB41908FBFD1080206961B1623C6C"
# }
#req = requests.get(url, headers=headers)
#response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 https://movie.naver.com/movie/sdb/browsing/bmovie.nhn?genre=19&page=1>

In [23]:
links = response.xpath('//*[@id="old_content"]/ul/li/a/@href').extract()
len(links)

20

In [24]:
link = response.urljoin(links[0])
link

'https://movie.naver.com/movie/bi/mi/basic.nhn?code=10157'

In [25]:
# 영화 상세데이터 수집

In [26]:
req = requests.get(link)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=10157>

In [27]:
title = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a/text()').extract()[0]
story = response.xpath('//*[@id="content"]/div[1]/div[4]/div[1]/div/div/p/text()').extract()[0]
ntz_star = response.xpath('//*[@id="pointNetizenPersentBasic"]/em/text()').extract()
ntz_star = "".join(ntz_star)
review = response.xpath('//*[@id="content"]/div[1]/div[4]/div[4]/div[2]/div[4]/ul/li[1]/div[2]/p/text()')[1].extract().strip()
title, story, ntz_star, review

('007 골드핑거',
 '영국 국적의 골드핑거(Auric Goldfinger: 거트 프로브 분)는 정식 금매매업자이며 국제 보석상이다. 그가 보유하고 있는 영국의 상당량의 금을 외국으로 반출시킨다는 정보는 있으나 증거가 없는 영국 은행에서는 007(James Bond: 숀 코넬리 분)에게 증거를 찾도록 M(M: 버나드 리 분)을 통해 임무를 내린다. 명령을 받은 007은 그에게 접근하여 두번의 골탕과 금전적 손실을 입히나 그 대가로 매스터슨(Jill Masterson: 셜리 엣톤 분 / Tilly Masterson: 타니아 말렛 분) 자매가 죽음을 당한다. 제네바까지 쫓아간 007은 골드핑거의 고급승용차인 롤스로이스 팬텀 3의 보디를 금으로 만들어 밀수한다는 사실을 알아낸다. 허나 그 와중에 007은 잡히게 되고 레이저의 위협도 받게 되는데.',
 '7.41',
 '역대 007영화 중 최고의 영화')

In [34]:
%%writefile navermovie/navermovie/spiders/spider.py

import scrapy
import requests

from navermovie.items import NavermovieItem

class NavermovieSpider(scrapy.Spider):
    custom_settings = {
        "DOWNLOADER_MIDDLEWARES": {
            "scrapy.downloadmiddlewares.useragent.UserAgentMiddleware": None,
            "scrapy_fake_useragent.middleware.RandomUserAgentMiddleware": 400,
        }
    }
    
    name = "Navermovie"
    allow_domain = ["https://movie.naver.com"]
    start_urls = ["https://movie.naver.com/movie/sdb/browsing/bmovie.nhn?genre=19&page=1"]
    
    def parse(self, response):
        links = response.xpath('//*[@id="old_content"]/ul/li/a/@href').extract()
        links = list(map(lambda data:response.urljoin(data), links))
        # 페이지별 링크 만들기
        for link in links:
            link = response.urljoin(link)
            yield scrapy.Request(link, callback=self.parse_page)
            
    def parse_page(self, response):
        links = response.xpath('//*[@id="old_content"]/ul/li[1]/a/@href').extract()
        links = list(map(lambda data:response.urljoin(data), links))
        for link in links:
            yield scrapy.Request(link, callback=self.parse_page_contents)

    
   
    # 상세페이지에서 컨텐츠 가져오기            
    def parse_page_contents(self, response):
        item = NavermovieItem()
        item["title"] = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a/text()').extract()[0]
        item["story"] = response.xpath('//*[@id="content"]/div[1]/div[4]/div[1]/div/div/p/text()').extract()[0]
        item["ntz_star"] = response.xpath('//*[@id="pointNetizenPersentBasic"]/em/text()').extract()
        ntz_star = "".join(ntz_star)
        item["review"] = response.xpath('//*[@id="content"]/div[1]/div[4]/div[4]/div[2]/div[4]/ul/li[1]/div[2]/p/text()')[1].extract().strip()
        yield item

Overwriting navermovie/navermovie/spiders/spider.py


In [35]:
%%writefile run.sh
cd navermovie
scrapy crawl Navermovie -o navermovie01.csv

Overwriting run.sh


In [37]:
!source run.sh

2020-07-21 19:18:18 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: navermovie)
2020-07-21 19:18:18 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.7.4 (default, Aug 13 2019, 15:17:50) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.8, Platform Darwin-19.6.0-x86_64-i386-64bit
2020-07-21 19:18:18 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'navermovie', 'FEED_FORMAT': 'csv', 'FEED_URI': 'navermovie01.csv', 'NEWSPIDER_MODULE': 'navermovie.spiders', 'SPIDER_MODULES': ['navermovie.spiders']}
2020-07-21 19:18:18 [scrapy.extensions.telnet] INFO: Telnet Password: 465ed8163ac6d806
2020-07-21 19:18:18 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions

In [36]:
!head -n 25 navermovie/navermovie/settings.py | tail -n 5

# Obey robots.txt rules
ROBOTSTXT_OBEY = False

# Configure maximum concurrent requests performed by Scrapy (default: 16)
#CONCURRENT_REQUESTS = 32


In [71]:
!sed -i 's/ROBOTSTXT_OBEY = True/ROBOTSTXT_OBEY = False/' navermovie/navermovie/settings.py

sed: 1: "navermovie/navermovie/s ...": extra characters at the end of n command


In [39]:
df = pd.read_csv("navermovie/navermovie01.csv")
df.tail(2)

EmptyDataError: No columns to parse from file